<a href="https://colab.research.google.com/github/leiyi-ihb/ColabPractics/blob/main/lyProject/03_05_quiz_modelOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 将loss='binary_crossentropy'调整为loss='msa'，可以发现模型一下就能达到很好
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPooling1D, GlobalMaxPooling1D, Conv1D, Flatten, AveragePooling1D, GlobalAveragePooling1D
from keras.optimizers import Adam, SGD

In [3]:
! if [ ! -f "dataset_Li.pkl" ]; then wget https://master.dl.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl ; else echo "Found previous downloaded data."; fi

--2022-11-03 18:31:47--  https://master.dl.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl
Resolving master.dl.sourceforge.net (master.dl.sourceforge.net)... 216.105.38.12
Connecting to master.dl.sourceforge.net (master.dl.sourceforge.net)|216.105.38.12|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl [following]
--2022-11-03 18:31:48--  https://downloads.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Connecting to downloads.sourceforge.net (downloads.sourceforge.net)|204.68.111.105|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://versaweb.dl.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl [following]
--2022-11-03 18:31:48--  https://versaweb.dl.sourceforge.net/project/summer-re

In [4]:
# Data processing
import pickle
with open("dataset_Li.pkl", "rb") as f:
    data = pickle.load(f)
x_train, y_train = np.array(data['train'][0]), np.array(data['train'][1])
x_test, y_test = np.array(data['test'][0]), np.array(data['test'][1])

print(x_train.shape)

(16000, 1000, 4)


In [6]:

class Model:  
  def __init__(self, learnRate):        
    '''optimizer'''
    self.adam = Adam(lr=learnRate)
    self.sgd = SGD(lr=learnRate) 
    self.model = Sequential()

  def modelBuild(self, filter, poolSize, kernelSize, ConvAct, DenAct, input):
    model = self.model
    '''model build'''   
    model.add(Conv1D(filters=filter, kernel_size=(kernelSize), activation=ConvAct, input_shape=input))
    model.add(MaxPooling1D(pool_size=(poolSize)))
    model.add(GlobalMaxPooling1D())
    model.add(Flatten())
    model.add(Dense(32, activation=DenAct))
    model.add(Dense(1, activation=DenAct))
    print(model.summary())

  def modelCompile(self, met, los):
    '''model compile'''
    model = self.model
    model.compile(optimizer=self.adam,
             metrics = [met],
             loss = los)

  def modelFit(self, xTrain, yTrain, epo, batSize): 
    '''model fix'''
    model = self.model 
    model.fit(xTrain, 
              yTrain,
              epochs = epo,
              batch_size = batSize
              )
    
  def modelEvaluate(self, xTrain, yTrain, xTest, yTest, batSize):
    '''model evaluate'''
    model = self.model
    loss, accuracy = model.evaluate(xTrain, yTrain, batch_size=batSize)
    print("\ntest loss:", loss)
    print('accuracy:', accuracy)

    loss, accuracy = model.evaluate(xTest, yTest) #将我们的训练集也评估一下
    print("\ntrain loss:", loss)
    print('accuracy:', accuracy)



if __name__=='__main__':
  '''
  1, Conv1d: Filters = {32, 16, 3}; Kernel_size = {20, 10, 7, 5, 3}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  2, Pooling1d: Max or Avg pool; Pool size = {8, 4, 2,}
  3, Flatten: Global Max, Global Avg, Flatten
  4, Dense: Units = {32, 64}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  '''
  kernel_size =[20, 10, 7, 5, 3]
  activation = ["relu", "tanh", "linear"]
  poolSize = [8, 4, 2]
  deAct = ["relu", "tanh", "linear"]

  # from itertools import product
  # loop_val = [kernel_size, activation, poolSize, deAct] #
  # for i in product(*loop_val):
  #   print(x_train.shape)
  #   print(i)
  obj1 = Model(0.01)
    # obj1.modelBuild(filter=32, poolSize=i[2], kernelSize=i[0], ConvAct=i[1], DenAct=i[3], input=(1000, 4))
  obj1.modelBuild(filter=32, poolSize=8, kernelSize=20, ConvAct="relu", DenAct="relu", input=(1000, 4))
  obj1.modelCompile(met="acc", los='binary_crossentropy')
  obj1.modelFit(xTrain=x_train, yTrain=y_train, epo=10, batSize=512)
  obj1.modelEvaluate(xTrain=x_train, yTrain=y_train, xTest=x_test, yTest=y_test, batSize=512)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 981, 32)           2592      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 122, 32)          0         
 1D)                                                             
                                                                 
 global_max_pooling1d_3 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 flatten_3 (Flatten)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 32)                1056      
                                                                 
 dense_7 (Dense)             (None, 1)                

In [14]:

class Model:  
  def __init__(self, learnRate):        
    '''optimizer'''
    self.adam = Adam(lr=learnRate, decay=1e-6)
    self.sgd = SGD(lr=learnRate) 
    self.model = Sequential()

  def modelBuild(self, filter, poolSize, kernelSize, ConvAct, DenAct, input):
    model = self.model
    '''model build'''   
    model.add(Conv1D(filters=filter, kernel_size=(kernelSize), activation=ConvAct, input_shape=input))
    model.add(MaxPooling1D(pool_size=(poolSize)))
    model.add(GlobalMaxPooling1D())
    model.add(Flatten())
    model.add(Dense(32, activation=DenAct))
    model.add(Dense(1, activation=DenAct))
    print(model.summary())

  def modelCompile(self, met, los):
    '''model compile'''
    model = self.model
    model.compile(optimizer=self.adam,
             metrics = [met],
             loss = los)

  def modelFit(self, xTrain, yTrain, epo, batSize): 
    '''model fix'''
    model = self.model 
    model.fit(xTrain, 
              yTrain,
              epochs = epo,
              batch_size = batSize
              )
    
  def modelEvaluate(self, xTrain, yTrain, xTest, yTest, batSize):
    '''model evaluate'''
    model = self.model
    loss, accuracy = model.evaluate(xTrain, yTrain, batch_size=batSize)
    print("\ntest loss:", loss)
    print('accuracy:', accuracy)

    loss, accuracy = model.evaluate(xTest, yTest) #将我们的训练集也评估一下
    print("\ntrain loss:", loss)
    print('accuracy:', accuracy)



if __name__=='__main__':
  '''
  1, Conv1d: Filters = {32, 16, 3}; Kernel_size = {20, 10, 7, 5, 3}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  2, Pooling1d: Max or Avg pool; Pool size = {8, 4, 2,}
  3, Flatten: Global Max, Global Avg, Flatten
  4, Dense: Units = {32, 64}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  '''
  kernel_size =[20, 10, 7, 5, 3]
  activation = ["relu", "tanh", "linear"]
  poolSize = [8, 4, 2]
  deAct = ["relu", "tanh", "linear"]

  # from itertools import product
  # loop_val = [kernel_size, activation, poolSize, deAct] #
  # for i in product(*loop_val):
  #   print(x_train.shape)
  #   print(i)
  obj1 = Model(0.001)
    # obj1.modelBuild(filter=32, poolSize=i[2], kernelSize=i[0], ConvAct=i[1], DenAct=i[3], input=(1000, 4))
  obj1.modelBuild(filter=32, poolSize=8, kernelSize=20, ConvAct="relu", DenAct="relu", input=(1000, 4))
  obj1.modelCompile(met="acc", los='mse')
  obj1.modelFit(xTrain=x_train, yTrain=y_train, epo=10, batSize=512)
  obj1.modelEvaluate(xTrain=x_train, yTrain=y_train, xTest=x_test, yTest=y_test, batSize=512)


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_11 (Conv1D)          (None, 981, 32)           2592      
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 122, 32)          0         
 g1D)                                                            
                                                                 
 global_max_pooling1d_11 (Gl  (None, 32)               0         
 obalMaxPooling1D)                                               
                                                                 
 flatten_11 (Flatten)        (None, 32)                0         
                                                                 
 dense_22 (Dense)            (None, 32)                1056      
                                                                 
 dense_23 (Dense)            (None, 1)               